# ⚙️ Fraud Detection Pipeline - Part 3: Feature Engineering

## 📋 Overview
This notebook covers:
- Loading preprocessed data
- Creating new features from existing ones
- Feature transformations (log, polynomial)
- Feature scaling and normalization
- Feature selection techniques
- Train/Validation/Test split

---

## 1️⃣ Setup & Load Data

In [2]:
import os
BASE_PATH = os.path.abspath('.')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.ensemble import RandomForestClassifier

import pickle
import json
import warnings
import gc
warnings.filterwarnings('ignore')

COLORS = {
    'primary': '#3498db',
    'secondary': '#2ecc71',
    'danger': '#e74c3c',
    'warning': '#f39c12',
    'info': '#9b59b6'
}

print('✅ Libraries imported!')

✅ Libraries imported!


In [4]:
train_df = pd.read_pickle(f'{BASE_PATH}/data/processed/train_preprocessed.pkl')
test_df = pd.read_pickle(f'{BASE_PATH}/data/processed/test_preprocessed.pkl')

with open(f'{BASE_PATH}/reports/metrics/preprocessing_info.json', 'r') as f:
    prep_info = json.load(f)

TARGET_COL = prep_info['target_column']
NUMERIC_COLS = prep_info['numeric_columns']
CATEGORICAL_COLS = prep_info['categorical_columns']

print(f'📊 Train shape: {train_df.shape}')
print(f'📊 Test shape: {test_df.shape}')
print(f'🎯 Target: {TARGET_COL}')

📊 Train shape: (590540, 392)
📊 Test shape: (506691, 391)
🎯 Target: isFraud


## 2️⃣ Create New Features

In [5]:
def create_features(df):
    """
    Create new features from existing columns
    """
    df = df.copy()
    
    if 'TransactionAmt' in df.columns:
        df['TransactionAmt_Log'] = np.log1p(df['TransactionAmt'])
        df['TransactionAmt_Decimal'] = df['TransactionAmt'] - np.floor(df['TransactionAmt'])
        df['TransactionAmt_IsRound'] = (df['TransactionAmt'] % 1 == 0).astype(int)
        df['TransactionAmt_Cents'] = ((df['TransactionAmt'] * 100) % 100).astype(int)
    
    if 'TransactionDT' in df.columns:
        df['TransactionDT_Hour'] = (df['TransactionDT'] // 3600) % 24
        df['TransactionDT_DayOfWeek'] = (df['TransactionDT'] // 86400) % 7
        df['TransactionDT_IsWeekend'] = (df['TransactionDT_DayOfWeek'] >= 5).astype(int)
        df['TransactionDT_IsNight'] = ((df['TransactionDT_Hour'] >= 22) | (df['TransactionDT_Hour'] <= 6)).astype(int)
    
    card_cols = [c for c in df.columns if c.startswith('card')]
    if len(card_cols) >= 2:
        df['Card_Combination'] = df[card_cols[0]].astype(str) + '_' + df[card_cols[1]].astype(str)
        df['Card_Combination'] = df['Card_Combination'].apply(lambda x: hash(x) % 10000)
    
    if 'addr1' in df.columns and 'addr2' in df.columns:
        df['Addr_Match'] = (df['addr1'] == df['addr2']).astype(int)
    
    if 'dist1' in df.columns:
        df['dist1_Log'] = np.log1p(df['dist1'].clip(0))
    if 'dist2' in df.columns:
        df['dist2_Log'] = np.log1p(df['dist2'].clip(0))
    
    c_cols = [c for c in df.columns if c.startswith('C') and c[1:].isdigit()]
    if len(c_cols) > 0:
        df['C_Sum'] = df[c_cols].sum(axis=1)
        df['C_Mean'] = df[c_cols].mean(axis=1)
        df['C_Std'] = df[c_cols].std(axis=1)
        df['C_Max'] = df[c_cols].max(axis=1)
        df['C_Min'] = df[c_cols].min(axis=1)
    
    d_cols = [c for c in df.columns if c.startswith('D') and c[1:].isdigit()]
    if len(d_cols) > 0:
        df['D_Sum'] = df[d_cols].sum(axis=1)
        df['D_Mean'] = df[d_cols].mean(axis=1)
        df['D_Std'] = df[d_cols].std(axis=1)
    
    v_cols = [c for c in df.columns if c.startswith('V') and c[1:].isdigit()]
    if len(v_cols) > 10:
        v_subset = v_cols[:50]
        df['V_Sum'] = df[v_subset].sum(axis=1)
        df['V_Mean'] = df[v_subset].mean(axis=1)
        df['V_Std'] = df[v_subset].std(axis=1)
    
    if 'P_emaildomain' in df.columns:
        df['Email_IsPopular'] = df['P_emaildomain'].isin([0, 1, 2, 3, 4]).astype(int)
    
    if 'TransactionAmt' in df.columns and 'TransactionDT_Hour' in df.columns:
        df['Amount_Hour_Interaction'] = df['TransactionAmt_Log'] * df['TransactionDT_Hour']
    
    return df

In [6]:
print('⚙️ Creating new features...')

original_train_cols = train_df.shape[1]
original_test_cols = test_df.shape[1]

train_df = create_features(train_df)
test_df = create_features(test_df)

new_train_cols = train_df.shape[1] - original_train_cols
new_test_cols = test_df.shape[1] - original_test_cols

print(f'✅ Created {new_train_cols} new features in train')
print(f'✅ Created {new_test_cols} new features in test')
print(f'\n📊 New train shape: {train_df.shape}')
print(f'📊 New test shape: {test_df.shape}')

⚙️ Creating new features...
✅ Created 24 new features in train
✅ Created 24 new features in test

📊 New train shape: (590540, 416)
📊 New test shape: (506691, 415)


In [7]:
new_features = [
    'TransactionAmt_Log', 'TransactionAmt_Decimal', 'TransactionAmt_IsRound', 'TransactionAmt_Cents',
    'TransactionDT_Hour', 'TransactionDT_DayOfWeek', 'TransactionDT_IsWeekend', 'TransactionDT_IsNight',
    'C_Sum', 'C_Mean', 'C_Std', 'D_Sum', 'D_Mean', 'V_Sum', 'V_Mean'
]

new_features_present = [f for f in new_features if f in train_df.columns]

print('📋 New Features Created:')
for f in new_features_present:
    print(f'  ✓ {f}')

📋 New Features Created:
  ✓ TransactionAmt_Log
  ✓ TransactionAmt_Decimal
  ✓ TransactionAmt_IsRound
  ✓ TransactionAmt_Cents
  ✓ TransactionDT_Hour
  ✓ TransactionDT_DayOfWeek
  ✓ TransactionDT_IsWeekend
  ✓ TransactionDT_IsNight
  ✓ C_Sum
  ✓ C_Mean
  ✓ C_Std
  ✓ D_Sum
  ✓ D_Mean
  ✓ V_Sum
  ✓ V_Mean


## 3️⃣ Analyze New Features

In [8]:
new_features_corr = train_df[new_features_present + [TARGET_COL]].corr()[TARGET_COL].drop(TARGET_COL)
new_features_corr = new_features_corr.sort_values(key=abs, ascending=False)

print('🔗 New Features Correlation with Fraud:')
for feat, corr in new_features_corr.items():
    direction = '↑' if corr > 0 else '↓'
    print(f'  {feat}: {corr:+.4f} {direction}')

🔗 New Features Correlation with Fraud:
  V_Mean: +0.1362 ↑
  V_Sum: +0.1362 ↑
  D_Mean: -0.0773 ↓
  D_Sum: -0.0773 ↓
  TransactionAmt_Cents: -0.0488 ↓
  TransactionAmt_Decimal: -0.0485 ↓
  TransactionDT_IsNight: +0.0122 ↑
  TransactionDT_Hour: -0.0118 ↓
  TransactionDT_DayOfWeek: -0.0077 ↓
  TransactionDT_IsWeekend: -0.0037 ↓
  C_Mean: +0.0033 ↑
  C_Sum: +0.0033 ↑
  TransactionAmt_IsRound: +0.0015 ↑
  C_Std: +0.0006 ↑
  TransactionAmt_Log: -0.0006 ↓


In [9]:
colors = [COLORS['danger'] if x > 0 else COLORS['primary'] for x in new_features_corr.values]

fig = go.Figure(go.Bar(
    x=new_features_corr.values,
    y=new_features_corr.index,
    orientation='h',
    marker_color=colors,
    text=[f'{x:.4f}' for x in new_features_corr.values],
    textposition='outside'
))

fig.update_layout(
    title='🔗 New Features Correlation with Fraud',
    title_font_size=18,
    xaxis_title='Correlation',
    yaxis_title='Feature',
    height=500,
    margin=dict(l=200)
)

fig.show()
fig.write_html(f'{BASE_PATH}/reports/figures/09_new_features_correlation.html')

In [10]:
if 'TransactionDT_Hour' in train_df.columns:
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Fraud Rate by Hour', 'Transaction Volume by Hour')
    )
    
    hourly_fraud = train_df.groupby('TransactionDT_Hour')[TARGET_COL].mean() * 100
    
    fig.add_trace(
        go.Scatter(
            x=hourly_fraud.index,
            y=hourly_fraud.values,
            mode='lines+markers',
            marker=dict(size=10, color=COLORS['danger']),
            line=dict(width=3),
            name='Fraud Rate'
        ),
        row=1, col=1
    )
    
    hourly_volume = train_df.groupby('TransactionDT_Hour').size()
    
    fig.add_trace(
        go.Bar(
            x=hourly_volume.index,
            y=hourly_volume.values,
            marker_color=COLORS['primary'],
            name='Volume'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title='⏰ Time-Based Fraud Patterns',
        title_font_size=18,
        height=400,
        showlegend=False
    )
    
    fig.update_xaxes(title_text='Hour of Day', row=1, col=1)
    fig.update_xaxes(title_text='Hour of Day', row=1, col=2)
    fig.update_yaxes(title_text='Fraud Rate (%)', row=1, col=1)
    fig.update_yaxes(title_text='Transaction Count', row=1, col=2)
    
    fig.show()
    fig.write_html(f'{BASE_PATH}/reports/figures/10_time_patterns.html')

## 4️⃣ Feature Selection

In [11]:
feature_cols = [c for c in train_df.columns if c != TARGET_COL]

X = train_df[feature_cols]
y = train_df[TARGET_COL]

print(f'📊 Features for selection: {len(feature_cols)}')

📊 Features for selection: 415


In [12]:
print('🔧 Applying Variance Threshold...')

var_selector = VarianceThreshold(threshold=0.01)
var_selector.fit(X)

low_variance_cols = [col for col, selected in zip(feature_cols, var_selector.get_support()) if not selected]
high_variance_cols = [col for col, selected in zip(feature_cols, var_selector.get_support()) if selected]

print(f'  Low variance columns removed: {len(low_variance_cols)}')
print(f'  Remaining columns: {len(high_variance_cols)}')

🔧 Applying Variance Threshold...
  Low variance columns removed: 51
  Remaining columns: 364


In [13]:
print('\n🔧 Removing highly correlated features...')

def remove_correlated_features(df, threshold=0.95):
    """Remove features that are highly correlated with each other"""
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return to_drop

X_filtered = X[high_variance_cols]
correlated_cols = remove_correlated_features(X_filtered, threshold=0.95)

print(f'  Highly correlated columns removed: {len(correlated_cols)}')

remaining_cols = [c for c in high_variance_cols if c not in correlated_cols]
print(f'  Remaining columns: {len(remaining_cols)}')


🔧 Removing highly correlated features...
  Highly correlated columns removed: 95
  Remaining columns: 269


In [14]:
print('\n🔧 Applying SelectKBest (ANOVA F-score)...')

X_remaining = X[remaining_cols]

k_features = min(100, len(remaining_cols))

selector = SelectKBest(score_func=f_classif, k=k_features)
selector.fit(X_remaining, y)

selected_mask = selector.get_support()
selected_features = [col for col, selected in zip(remaining_cols, selected_mask) if selected]

feature_scores = pd.DataFrame({
    'Feature': remaining_cols,
    'F_Score': selector.scores_,
    'P_Value': selector.pvalues_
}).sort_values('F_Score', ascending=False)

print(f'  Selected top {k_features} features')
print(f'\nTop 20 features by F-score:')
feature_scores.head(20)


🔧 Applying SelectKBest (ANOVA F-score)...
  Selected top 100 features

Top 20 features by F-score:


,Feature,F_Score,P_Value
190,V258,71120.065715,0.0
185,V244,56158.054052,0.0
187,V246,54268.544816,0.0
183,V242,53734.380358,0.0
184,V243,52000.108821,0.0
189,V257,50459.317213,0.0
147,V187,48948.408668,0.0
149,V190,47312.285341,0.0
151,V199,44968.488120,0.0
146,V186,44738.064106,0.0


In [15]:
top_20 = feature_scores.head(20)

fig = go.Figure(go.Bar(
    x=top_20['F_Score'],
    y=top_20['Feature'],
    orientation='h',
    marker=dict(
        color=top_20['F_Score'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='F-Score')
    ),
    text=[f'{x:.1f}' for x in top_20['F_Score']],
    textposition='outside'
))

fig.update_layout(
    title='📊 Top 20 Features by ANOVA F-Score',
    title_font_size=18,
    xaxis_title='F-Score',
    yaxis_title='Feature',
    height=600,
    margin=dict(l=200)
)

fig.show()
fig.write_html(f'{BASE_PATH}/reports/figures/11_feature_importance_anova.html')

In [16]:
print('\n🔧 Computing Random Forest Feature Importance...')

sample_size = min(50000, len(X_remaining))
sample_idx = np.random.choice(len(X_remaining), sample_size, replace=False)

X_sample = X_remaining.iloc[sample_idx]
y_sample = y.iloc[sample_idx]

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_sample, y_sample)

rf_importance = pd.DataFrame({
    'Feature': remaining_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print(f'\nTop 20 features by RF Importance:')
rf_importance.head(20)


🔧 Computing Random Forest Feature Importance...

Top 20 features by RF Importance:


,Feature,Importance
190,V258,0.049714
13,C1,0.033699
185,V244,0.028542
148,V188,0.020313
266,V_Sum,0.019438
183,V242,0.019215
149,V190,0.018090
19,C12,0.017941
14,C4,0.017782
69,V45,0.016407


In [17]:
top_20_rf = rf_importance.head(20)

fig = go.Figure(go.Bar(
    x=top_20_rf['Importance'],
    y=top_20_rf['Feature'],
    orientation='h',
    marker=dict(
        color=top_20_rf['Importance'],
        colorscale='Plasma',
        showscale=True,
        colorbar=dict(title='Importance')
    ),
    text=[f'{x:.4f}' for x in top_20_rf['Importance']],
    textposition='outside'
))

fig.update_layout(
    title='🌲 Top 20 Features by Random Forest Importance',
    title_font_size=18,
    xaxis_title='Importance',
    yaxis_title='Feature',
    height=600,
    margin=dict(l=200)
)

fig.show()
fig.write_html(f'{BASE_PATH}/reports/figures/12_feature_importance_rf.html')

In [18]:
print('\n🔧 Combining feature selection methods...')

top_n = 60
top_anova = set(feature_scores.head(top_n)['Feature'])
top_rf = set(rf_importance.head(top_n)['Feature'])

final_features = list(top_anova.union(top_rf))

print(f'  Top {top_n} from ANOVA: {len(top_anova)}')
print(f'  Top {top_n} from RF: {len(top_rf)}')
print(f'  Overlap: {len(top_anova.intersection(top_rf))}')
print(f'  Final feature set: {len(final_features)}')


🔧 Combining feature selection methods...
  Top 60 from ANOVA: 60
  Top 60 from RF: 60
  Overlap: 29
  Final feature set: 91


## 5️⃣ Feature Scaling

In [19]:
X_final = train_df[final_features]
X_test_final = test_df[[c for c in final_features if c in test_df.columns]]

missing_in_test = set(final_features) - set(X_test_final.columns)
if missing_in_test:
    print(f'⚠️ Features missing in test: {missing_in_test}')
    final_features = [c for c in final_features if c in X_test_final.columns]
    X_final = train_df[final_features]
    X_test_final = test_df[final_features]

print(f'📊 Final feature count: {len(final_features)}')

📊 Final feature count: 91


In [20]:
print('🔧 Applying RobustScaler...')

scaler = RobustScaler()

X_scaled = scaler.fit_transform(X_final)
X_test_scaled = scaler.transform(X_test_final)

X_scaled_df = pd.DataFrame(X_scaled, columns=final_features, index=X_final.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=final_features, index=X_test_final.index)

print('✅ Scaling complete!')
print(f'\n📊 Scaled train shape: {X_scaled_df.shape}')
print(f'📊 Scaled test shape: {X_test_scaled_df.shape}')

🔧 Applying RobustScaler...
✅ Scaling complete!

📊 Scaled train shape: (590540, 91)
📊 Scaled test shape: (506691, 91)


In [21]:
with open(f'{BASE_PATH}/models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print('💾 Scaler saved!')

💾 Scaler saved!


In [22]:
sample_features = final_features[:5]

fig = make_subplots(
    rows=2, cols=len(sample_features),
    subplot_titles=[f'{f} (Original)' for f in sample_features] + [f'{f} (Scaled)' for f in sample_features]
)

for i, feat in enumerate(sample_features):
    fig.add_trace(
        go.Histogram(x=X_final[feat].values[:10000], marker_color=COLORS['primary'], showlegend=False),
        row=1, col=i+1
    )
    fig.add_trace(
        go.Histogram(x=X_scaled_df[feat].values[:10000], marker_color=COLORS['secondary'], showlegend=False),
        row=2, col=i+1
    )

fig.update_layout(
    title='📊 Feature Scaling Effect (Before vs After)',
    title_font_size=18,
    height=500
)

fig.show()
fig.write_html(f'{BASE_PATH}/reports/figures/13_scaling_effect.html')

## 6️⃣ Train/Validation/Test Split

In [23]:
print('🔧 Splitting data...')

y = train_df[TARGET_COL]

X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled_df, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_val, X_holdout, y_val, y_holdout = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

print(f'\n📊 Split Summary:')
print(f'  Train:      {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X_scaled_df)*100:.1f}%)')
print(f'  Validation: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X_scaled_df)*100:.1f}%)')
print(f'  Holdout:    {X_holdout.shape[0]:,} samples ({X_holdout.shape[0]/len(X_scaled_df)*100:.1f}%)')
print(f'  Test (comp):{X_test_scaled_df.shape[0]:,} samples')

🔧 Splitting data...

📊 Split Summary:
  Train:      472,432 samples (80.0%)
  Validation: 59,054 samples (10.0%)
  Holdout:    59,054 samples (10.0%)
  Test (comp):506,691 samples


In [24]:
print('\n📊 Fraud Rate by Split:')
print(f'  Train:      {y_train.mean()*100:.2f}%')
print(f'  Validation: {y_val.mean()*100:.2f}%')
print(f'  Holdout:    {y_holdout.mean()*100:.2f}%')
print(f'  Original:   {y.mean()*100:.2f}%')


📊 Fraud Rate by Split:
  Train:      3.50%
  Validation: 3.50%
  Holdout:    3.50%
  Original:   3.50%


In [25]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}]],
    subplot_titles=('Dataset Split', 'Fraud Rate by Split')
)

fig.add_trace(
    go.Pie(
        labels=['Train', 'Validation', 'Holdout'],
        values=[len(X_train), len(X_val), len(X_holdout)],
        hole=0.4,
        marker_colors=[COLORS['primary'], COLORS['secondary'], COLORS['warning']]
    ),
    row=1, col=1
)

splits = ['Train', 'Validation', 'Holdout', 'Original']
fraud_rates = [y_train.mean()*100, y_val.mean()*100, y_holdout.mean()*100, y.mean()*100]

fig.add_trace(
    go.Bar(
        x=splits,
        y=fraud_rates,
        marker_color=[COLORS['primary'], COLORS['secondary'], COLORS['warning'], COLORS['info']],
        text=[f'{r:.2f}%' for r in fraud_rates],
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title='📊 Data Split Summary',
    title_font_size=18,
    height=400,
    showlegend=False
)

fig.show()
fig.write_html(f'{BASE_PATH}/reports/figures/14_data_split.html')

## 7️⃣ Save Processed Data

In [26]:
np.save(f'{BASE_PATH}/data/splits/X_train.npy', X_train.values)
np.save(f'{BASE_PATH}/data/splits/X_val.npy', X_val.values)
np.save(f'{BASE_PATH}/data/splits/X_holdout.npy', X_holdout.values)
np.save(f'{BASE_PATH}/data/splits/X_test.npy', X_test_scaled_df.values)

np.save(f'{BASE_PATH}/data/splits/y_train.npy', y_train.values)
np.save(f'{BASE_PATH}/data/splits/y_val.npy', y_val.values)
np.save(f'{BASE_PATH}/data/splits/y_holdout.npy', y_holdout.values)

print('💾 Numpy arrays saved!')

💾 Numpy arrays saved!


In [27]:
feature_engineering_info = {
    'final_features': final_features,
    'n_features': len(final_features),
    'train_shape': list(X_train.shape),
    'val_shape': list(X_val.shape),
    'holdout_shape': list(X_holdout.shape),
    'test_shape': list(X_test_scaled_df.shape),
    'target_column': TARGET_COL,
    'fraud_rate_train': float(y_train.mean()),
    'fraud_rate_val': float(y_val.mean())
}

with open(f'{BASE_PATH}/reports/metrics/feature_engineering_info.json', 'w') as f:
    json.dump(feature_engineering_info, f, indent=2)

feature_scores.to_csv(f'{BASE_PATH}/reports/metrics/feature_scores_anova.csv', index=False)
rf_importance.to_csv(f'{BASE_PATH}/reports/metrics/feature_scores_rf.csv', index=False)

print('💾 Feature info saved!')

💾 Feature info saved!
